<a href="https://colab.research.google.com/github/Sayed-Ali-Raza-Naqvi/ITSOLERA_AI_Skin_Conditions_Advisor_Project/blob/main/ITSOLERA_AI_Skin_Conditions_Advisor_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
from zipfile import ZipFile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
file_name = "Skin_Conditions_Aug.zip"

In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from zipfile import ZipFile

file_path_in_drive = '/content/drive/MyDrive/Skin_Conditions_Aug.zip'
with ZipFile(file_path_in_drive, 'r') as zip:
    zip.extractall()
    print('Done')

Done


In [ ]:
class_names = ['Acne', 'Carcinoma', 'Eczema', 'Keratosis', 'Milia', 'Rosacea']

In [ ]:
base_dir = '/content/Skin_Conditions'
batch_size = 32
img_size = (224, 224)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest',
    validation_split=0.2
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 1920 images belonging to 6 classes.


In [ ]:
validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 474 images belonging to 6 classes.


In [ ]:
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step


In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
for layer in base_model.layers[-100:]:
    layer.trainable = True

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint('inception_resnet_v2.keras', monitor='val_loss', save_best_only=True, mode='min')
early_stop = EarlyStopping(monitor='val_loss', patience=5, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=15,
    callbacks=[checkpoint, early_stop, reduce_lr]
)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


60/60 ━━━━━━━━━━━━━━━━━━━━ 2232s 35s/step - accuracy: 0.5220 - loss: 1.2587 - val_accuracy: 0.7924 - val_loss: 0.7023 - learning_rate: 1.0000e-04
Epoch 2/15


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


60/60 ━━━━━━━━━━━━━━━━━━━━ 17s 279ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8462 - val_loss: 0.6388 - learning_rate: 1.0000e-04
Epoch 3/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 2091s 35s/step - accuracy: 0.8900 - loss: 0.3325 - val_accuracy: 0.8371 - val_loss: 0.4913 - learning_rate: 1.0000e-04
Epoch 4/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 8s 132ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8462 - val_loss: 0.5112 - learning_rate: 1.0000e-04
Epoch 5/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 2094s 34s/step - accuracy: 0.9361 - loss: 0.1869 - val_accuracy: 0.8661 - val_loss: 0.4757 - learning_rate: 1.0000e-04
Epoch 6/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 94ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8846 - val_loss: 0.5076 - learning_rate: 1.0000e-04
Epoch 7/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 2140s 35s/step - accuracy: 0.9363 - loss: 0.1696 - val_accuracy: 0.8973 - val_loss: 0.4079 - learning_rate: 1.0000e-04
Epoch 8/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 41s 677ms/

In [ ]:
loss, accuracy = model.evaluate(validation_generator)
print(f'\nValidation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')

15/15 ━━━━━━━━━━━━━━━━━━━━ 127s 8s/step - accuracy: 0.9193 - loss: 0.3568

Validation Loss: 0.30350932478904724
Validation Accuracy: 0.9261603355407715


In [ ]:
y_true = validation_generator.classes
y_pred = model.predict(validation_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

15/15 ━━━━━━━━━━━━━━━━━━━━ 136s 9s/step


In [ ]:
cm = confusion_matrix(y_true, y_pred_classes)
cm

array([[17, 11, 13, 11, 15, 12],
       [17, 12, 14, 10,  8, 18],
       [ 9, 15, 14, 18, 11, 12],
       [12, 16,  8, 12, 17, 14],
       [10, 20, 12, 13, 14, 10],
       [12, 13, 12, 13, 14, 15]])

In [ ]:
report = classification_report(y_true, y_pred_classes, target_names=class_names)
report

'              precision    recall  f1-score   support\n\n        Acne       0.22      0.22      0.22        79\n   Carcinoma       0.14      0.15      0.14        79\n      Eczema       0.19      0.18      0.18        79\n   Keratosis       0.16      0.15      0.15        79\n       Milia       0.18      0.18      0.18        79\n     Rosacea       0.19      0.19      0.19        79\n\n    accuracy                           0.18       474\n   macro avg       0.18      0.18      0.18       474\nweighted avg       0.18      0.18      0.18       474\n'

In [ ]:
from google.colab import files
files.download('inception_resnet_v2.keras')
